# Rough example usage

In [1]:
%load_ext autoreload
%autoreload 2

from blackboxbandits import compare, bandits, meta, utils
import pandas as pd

pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:20,.4f}'.format

## Base optimizer comparison

### Run a new comparison

In [ ]:
import pmlb
dataset_names = list(map(lambda x: "clf-"+x.replace("_","-"), pmlb.classification_dataset_names)) \
                + list(map(lambda x: "reg-"+x.replace("_","-"), pmlb.regression_dataset_names))

In [ ]:
base_comparison = compare.BaseOptimizerComparison(
    optimizers=["HyperOpt", "OpenTuner-BanditA", "OpenTuner-GA",
                "OpenTuner-GA-DE", "PySOT", "RandomSearch",
                "Scikit-GBRT-Hedge", "Scikit-GP-Hedge", "Scikit-GP-LCB"],
    classifiers=["MLP-adam","lasso"],
    datasets = dataset_names,
    metrics=["mse", "nll"],
    num_calls=20,
    num_repetitions=1,
    db_root = "./base_results",
    datasets_root="./penn_datasets",
    parallel = True
)

In [ ]:
base_comparison.run()
dbid = base_comparison.get_dbid()
dbid

### View data from a previous comparison

In [2]:
DBID = "bo_20220228_124924_b7rkpeqg"

In [ ]:
df = compare.BaseOptimizerComparison.get_results_for_dbid(db_root="./saved_results", dbid=DBID)

## Meta optimizer comparison

### Set up meta optimizers

In [ ]:
# factors = [(a,b) for a in range(1,7) for b in range(1,7) if a*b < 7]

# S = 1
# GAMMA = 0.2

# best = {
#     f"best_{T}": meta.BestFixedTAlgos(T=T)
#     for T in range(2,7)}
# fpml_fixedexp = {
#     f"fpml_fixedexp_{T}": meta.BanditMetaOptimizer(
#         bandit_type=bandits.FPMLFixed,
#         T=T, S=S)
#     for T in range(2,7)}
# fpml_probexp = {
#     f"fpml_probexp_{T}": meta.BanditMetaOptimizer(
#         bandit_type=bandits.FPMLProb,
#         T=T, gamma=GAMMA)
#     for T in range(2,7)}
# fpml_gr_noexp = {
#     f"fpml_gr_noexp_{T}": meta.BanditMetaOptimizer(
#         bandit_type=bandits.FPMLWithGR,
#         T=T, gamma=0)
#     for T in range(2,7)}
# fpml_gr_probexp = {
#     f"fpml_gr_probexp_{T}": meta.BanditMetaOptimizer(
#         bandit_type=bandits.FPMLWithGR,
#         T=T, gamma=GAMMA)
#     for T in range(2,7)}
# streeter_fpml = {
#     f"streeter_fpml_{a}x{b}": meta.BanditMetaOptimizer(
#         bandit_type=bandits.StreeterFPML,
#         T=a*b, T_1=a, T_2=b, gamma=GAMMA, gr=True)
#     for a,b in factors}
# streeter_exp3 = {
#     f"streeter_exp3_{T}": meta.BanditMetaOptimizer(
#         bandit_type=bandits.Streeter,
#         T=T)
#     for T in range(2,7)}

# meta_optimizers = {**fpml_fixedexp, **fpml_probexp, **fpml_gr_noexp,
#                    **fpml_gr_probexp, **streeter_fpml, **streeter_exp3}

In [3]:
S = 1
GAMMA = lambda T: 1/T

fpml_fixedexp = {
    f"fpml_fixedexp_{T}": meta.BanditMetaOptimizer(
        bandit_type=bandits.FPMLFixed,
        T=T, S=S)
    for T in range(2,3)}
# fpml_probexp = {
#     f"fpml_probexp_{T}": meta.BanditMetaOptimizer(
#         bandit_type=bandits.FPMLProb,
#         T=T, gamma=GAMMA(T))
#     for T in range(2,7)}
# fpml_gr_noexp = {
#     f"fpml_gr_noexp_{T}": meta.BanditMetaOptimizer(
#         bandit_type=bandits.FPMLWithGR,
#         T=T, gamma=0)
#     for T in range(2,7)}
# fpml_gr_probexp = {
#     f"fpml_gr_probexp_{T}": meta.BanditMetaOptimizer(
#         bandit_type=bandits.FPMLWithGR,
#         T=T, gamma=GAMMA(T))
#     for T in range(2,7)}

meta_optimizers = {**fpml_fixedexp}
# meta_optimizers = {**fpml_fixedexp, **fpml_probexp, **fpml_gr_noexp, **fpml_gr_probexp}

### Setup and run meta comparison

In [12]:
meta_comparison = compare.MetaOptimizerComparison \
                         .from_precomputed_base_comparison(
    dbid=DBID,
    meta_optimizers=meta_optimizers,
    db_root = "./base_results",
    parallel_meta = True,
    num_meta_repetitions = 10
)

In [13]:
meta_comparison.run_meta_comparison()
# results_mean, results_std = meta_comparison.full_results()
# summary_mean, summary_std = meta_comparison.summary()

# results_mean.to_csv("meta_results/fpml_results_mean.csv")
# results_std.to_csv("meta_results/fpml_results_std.csv")
# summary_mean.to_csv("meta_results/fpml_summary_mean.csv")
# summary_std.to_csv("meta_results/fpml_summary_std.csv")

Starting meta-comparison, repetition 1 of 10
--------------------------------------------
Running meta-optimizer fpml_fixedexp_2 on base study 1 of 1
Finished meta-comparison in 0.9329264163970947 seconds
Starting meta-comparison, repetition 2 of 10
--------------------------------------------
Running meta-optimizer fpml_fixedexp_2 on base study 1 of 1
Finished meta-comparison in 0.9171254634857178 seconds
Starting meta-comparison, repetition 3 of 10
--------------------------------------------
Running meta-optimizer fpml_fixedexp_2 on base study 1 of 1
Finished meta-comparison in 0.923633337020874 seconds
Starting meta-comparison, repetition 4 of 10
--------------------------------------------
Running meta-optimizer fpml_fixedexp_2 on base study 1 of 1
Finished meta-comparison in 0.9264392852783203 seconds
Starting meta-comparison, repetition 5 of 10
--------------------------------------------
Running meta-optimizer fpml_fixedexp_2 on base study 1 of 1
Finished meta-comparison in 0.9

In [14]:
df = meta_comparison.meta_results
df = df.reorder_levels(["optimizer","function", "study_id", "meta_rep"])
df = df.sort_values(df.index.names)
df

visible_score  \
optimizer       function                                           study_id meta_rep                        
fpml_fixedexp_2 MLP-adam_clf-GAMETES-Epistasis-2-Way-1000atts-0... 0        0                      0.7768   
                                                                            1                      0.7768   
                                                                            2                      0.7768   
                                                                            3                      0.7768   
                                                                            4                      0.7768   
                                                                            5                      0.7768   
                                                                            6                      0.7768   
                                                                            7                      0.7768   
                                                                            8                      0.7768   
                                                                            9                      0.7768   
                MLP-adam_clf-GAMETES-Epistasis-2-Way-20atts-0.1... 0        0                      0.0000   
                                                                            1                      0.0000   
                                                                            2                      0.0000   
                                                                            3                      0.0000   
                                                                            4                      0.0000   
                                                                            5                      0.0000   
                                                                            6                      0.0000   
                                                                            7                      0.0000   
                                                                            8                      0.0000   
                                                                            9                      0.0000   
                MLP-adam_clf-GAMETES-Epistasis-2-Way-20atts-0.4... 0        0                      0.3885   
                                                                            1                      0.3885   
                                                                            2                      0.3885   
                                                                            3                      0.3885   
                                                                            4                      0.3885   
                                                                            5                      0.3885   
                                                                            6                      0.3885   
                                                                            7                      0.3885   
                                                                            8                      0.3885   
                                                                            9                      0.3885   
                MLP-adam_clf-GAMETES-Heterogeneity-20atts-1600-... 0        0                      0.0000   
                                                                            1                      0.0000   
                                                                            2                      0.0000   
                                                                            3                      0.0000   
                                                                            4                      0.0000   
                                                           

In [26]:
summary_mean, summary_std = meta_comparison.summary()

In [28]:
summary_std

,visible_score,generalization_score
optimizer,,
fpml_fixedexp_2,0.0000,0.0000
